# Notebook: Train Model

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from transformers import set_seed
from ACD import train_ACD_model
from E2E import train_E2E_model
from OTE import train_OTE_model
from ACSA import train_ACSA_model
from TASD import train_TASD_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "end_2_end_absa" # "aspect_category", "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"
LLM_SAMPLING = "random"

## Settings

In [3]:
# Set seeds
torch.device(constants.DEVICE)
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)
set_seed(constants.RANDOM_SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning,
                        module="transformers.optimization")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, LLM_SAMPLING)

500 5
500 5


### Load Model

In [5]:
if TARGET == "aspect_category":
    results = train_ACD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [6]:
if TARGET == "aspect_category_sentiment":
    results = train_ACSA_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [7]:
if TARGET == "end_2_end_absa":
    results = train_E2E_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

{'text': 'Das Fleisch des Brauhaustellers war ungenießbar!', 'tags': [{'end': 11, 'start': 4, 'tag_with_polarity': 'FOOD-NEGATIVE', 'tag_with_polarity_and_type': 'FOOD-NEGATIVE-explicit', 'text': 'Fleisch', 'type': 'label-explicit', 'label': 'FOOD', 'polarity': 'NEGATIVE'}], 'id': '01897e6e-f8cf-44ae-83d0-6db6c86285fb'}
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0,

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
if TARGET == "aspect_term":
    results = train_OTE_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [ ]:
if TARGET == "target_aspect_sentiment_detection":
    results = train_TASD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

### Save Results

In [ ]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [ ]:
results

### Remove useless folders

In [ ]:
try:
    shutil.rmtree("outputs")
except:
    pass